<h1><center>
Toxic Comments Kaggle Competition Data Preperation
</center></h1>
<h2><center>
Preparing the Data to Use in Modeling
</center></h2>
<h3><center>
Marcel Colvin 912033961
</center></h3>

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
import nltk
from nltk import corpus
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer as wnl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import gensim
from sklearn.neural_network import MLPClassifier
import time
import glob

In [ ]:
train = pd.read_csv('Toxic_comments/train.csv')
test = pd.read_csv('Toxic_comments/test.csv')

In [ ]:
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

### Investigate the data

In [ ]:
print('Total comments: {}'.format(len(train)))
print('toxic: {}'.format(train[train['toxic'] > 0]['toxic'].count()))
print('severe_toxic: {}'.format(train[train['severe_toxic'] > 0]['severe_toxic'].count()))
print('obscene: {}'.format(train[train['obscene'] > 0]['obscene'].count()))
print('threat: {}'.format(train[train['threat'] > 0]['threat'].count()))
print('insult: {}'.format(train[train['insult'] > 0]['insult'].count()))
print('identity_hate: {}'.format(train[train['identity_hate'] > 0]['identity_hate'].count()))

In [ ]:
train['length'] = train['comment_text'].str.len()

In [ ]:
def lemmatize(word):
    lemmatized = wnl().lemmatize(word.lower())
    return word
def tokenize(text):
    new_text = re.sub("[^0-9a-zA-Z]", ' ', text)
    new_list = new_text.lower().split()
    stop = stopwords.words('english')
    words = [x for x in new_list if not x in stop]
    final = [lemmatize(x) for x in words]
    return final
train['tokens'] = train["comment_text"].apply(tokenize)
test['tokens'] = test['comment_text'].apply(tokenize)

In [ ]:
google_model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [ ]:
def get_average_vector(tokens, vector, k=300):
    if len(tokens)<1:
        return np.zeros(k)
    vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens]
    average_vec = np.divide(np.sum(vectorized, axis=0), len(vectorized))
    return average_vec

train["avg_word"] = list(train['tokens'].apply(lambda x: get_average_vector(x, google_model)))
test["avg_word"] = list(test['tokens'].apply(lambda x: get_average_vector(x, google_model)))

In [ ]:
train.to_csv("training_data.csv", index = False)
test.to_csv("test_data.csv", index = False)

## Cleaning NYT Comments for Data Analysis

In [ ]:
path = '/home/marcel/Github/Toxic_comments/nyt-comments/comments/'
nyt_comments = pd.DataFrame()
for file_name in glob.glob(path+'*.csv'):
    nyt_comments = nyt_comments.append(pd.read_csv(file_name)[['commentBody', 'commentID', "reportAbuseFlag", 'userLocation', 'userID']])

In [ ]:
nyt_comments["commentBody"] = nyt_comments['commentBody'].map(lambda word : text_cleaner(word))

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
x = [None]*len(nyt_comments)
for i in range(len(nyt_comments)):
     x[i] = sid.polarity_scores(nyt_comments.iloc[i]["commentBody"])

In [ ]:
nyt_comments['neg'] = [d['neg'] for d in x]
nyt_comments['neu'] = [d['neu'] for d in x]
nyt_comments['pos'] = [d['pos'] for d in x]
nyt_comments['compound'] = [d['compound'] for d in x]

In [ ]:
nyt_comments = nyt_comments[nyt_comments.commentBody.notnull()]

In [ ]:
nyt_comments.to_csv("nyt_commentsdf.csv", index = False)